In [2]:
from symmer.operators import PauliwordOp
from symmer.projection import ContextualSubspace
from symmer import QuantumState
from symmer.utils import stab_renyi_entropy
import sys
from scipy import sparse
import itertools

sys.path.append('../')

import numpy as np
from IPython.display import HTML
import time
from qiskit._accelerate.sparse_pauli_op import (
    ZXPaulis,
    to_matrix_sparse,
)

We'll start by generating a random state

In [7]:
random_state=QuantumState.haar_random(n_qubits=11)

Okay, now we'll calculate the true SRE. For an $n$ qubit state, this checks the expectation values of all $4^n$ Pauli strings.

In [9]:
t1=time.time()
m2_true=stab_renyi_entropy(random_state)
t2=time.time()
print(m2_true)
print(t2-t1)

9.001572717393591
1013.0134127140045


In [8]:
t1=time.time()
m2_sample=stab_renyi_entropy(random_state,sampling=True,n_samples=1e6,sampling_approach='Metropolis')
t2=time.time()
print(m2_sample)
print(t2-t1)

9.002695595178428
253.6866900920868
